In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

See the discussion here:

https://github.com/run-llama/llama_index/discussions/13431

You should use ReactAgentWorker - the function calling agent worker is only for LLMs with a specific API built-in for tool calling (this includes openai, anthropic, and mistral)



In [15]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from tavily import TavilyClient 


# we_search

def web_search(query: str) -> str:
    """
    Performs a web search using the Tavily API and returns the context string.

    Parameters:
    - query (str): The search query.

    Returns:
    - str: The context string from the Tavily API or an error message.
    """
    try:
        # Step 1: Instantiate the TavilyClient
        tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

        # Step 2: Execute the search query
        context = tavily_client.get_search_context(query=query)

        # Step 3: Return the context
        return f"**Web Search Context:**\n{context}"
    except Exception as e:
        return f"Error performing web search: {str(e)}"

# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

web_search_tool = FunctionTool.from_defaults(fn=web_search)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

# initialize llm
llm = Ollama(model="llama3", temperature=0, format="json")

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool, web_search_tool], llm=llm, verbose=True,
                             max_iterations=5)

In [16]:
response = agent.chat("What is a good restaurant in Rio Rancho, NM? Include hyperlinks in your response.")

> Running step f98d65b2-78b7-4cd9-8b2b-0d11dbb60d44. Step input: What is a good restaurant in Rio Rancho, NM? Include hyperlinks in your response.
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: web_search
Action Input: {'query': 'best restaurants in Rio Rancho, NM'}
Observation: **Web Search Context:**
"[\"{\\\"url\\\": \\\"https://www.yelp.com/search?cflt=restaurants&find_loc=Rio+Rancho,+NM\\\", \\\"content\\\": \\\"These are the best restaurants with outdoor seating in Rio Rancho, NM: M'tucci's Moderno. Whiptail. Turtle Mountain Brewing - South. The Range Caf\\\\u00e9. Namaste. People also liked: Restaurants For Lunch. Best Restaurants in Rio Rancho, NM - Whiptail, Upscale Rio, Restaurant Forty Nine Forty, Fuego 505, Lily and Liam Bistro, Dagmar's Strudel House\\\"}\", \"{\\\"url\\\": \\\"https://www.yelp.com/search?find_desc=restaurants&find_loc=Rio+Rancho,+NM\\\", \\\"content\\\": \\\"Top 10 Best Restaurants in Rio

In [17]:
print(response)

Based on the web search context, some popular restaurants in Rio Rancho, NM include M'tucci's Moderno, Whiptail, Turtle Mountain Brewing - South, The Range CafÃ, and Namaste. Additionally, OpenTable recommends El Pinto as one of the best lunch restaurants in Rio Rancho. Restaurantji also lists M'tucci's Moderno, Brew Lab 101 Beer & Cider Co., Slate Street Billiards, O Sushi, and Squeezed Juice Bar - Rio Rancho as some of their current favorites.
